In [13]:
import hydra
from omegaconf import DictConfig, OmegaConf

import sys
import os
# Add project root to path
sys.path.insert(0, os.path.abspath('../../../../'))

# sorrel imports
from sorrel.examples.leakyemotions.agents import LeakyEmotionsAgent, Wolf
from sorrel.examples.leakyemotions.entities import EmptyEntity
from sorrel.examples.leakyemotions.env import LeakyEmotionsEnv, ENTITY_LIST
from sorrel.examples.leakyemotions.world import LeakyEmotionsWorld
from sorrel.examples.leakyemotions.wolf_model import WolfModel
from sorrel.action.action_spec import ActionSpec
from sorrel.models.human_player import HumanPlayer, HumanObservation

In [14]:
def main():
    for spawn_prob in [0.001, 0.002, 0.003, 0.004]:
        config = OmegaConf.create({
            "experiment": {
                "epochs": 1,
                "max_turns": 50,
                "record_period": 50,
            },
            "model": {
                "agent_vision_radius": 3,
                "epsilon_decay": 0.0001,
            },
            "world": {
                "agents": 2,
                "height": 9,
                "width": 9,
                "layers": 3,
                "spawn_prob": spawn_prob,
                "wolves":0,
            },
        })

    world = LeakyEmotionsWorld(config=config, default_entity=EmptyEntity())
    env = LeakyEmotionsEnv(world, config)
    agents = env.agents

    observation_spec = HumanObservation(
        entity_list=ENTITY_LIST,
        full_view=True,
        env_dims=(config.world.height, config.world.width),
    )
    action_spec = ActionSpec(["up", "down", "left", "right"])
    agents[-2] = LeakyEmotionsAgent(
        observation_spec=observation_spec,
        action_spec=action_spec,
        model=HumanPlayer(
            input_size=(config.world.height, config.world.width, config.world.layers),
            action_space=action_spec.n_actions,
            memory_size=1,
        ),
    )

    env.override_agents(agents=agents)
    env.run_experiment()
    return env

In [15]:
experiment = main()

AttributeError: 'HumanPlayer' object has no attribute 'state_value'